In [1]:
from tuwnlp.dataset import PropagandaDataset
from tuwnlp.utils import Language
from transformers import DistilBertTokenizer, DistilBertModel
from torch.utils.data import DataLoader
import torch

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)} is available.")
else:
    print("No GPU available. Training will run on CPU.")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


GPU: NVIDIA RTX 2000 Ada Generation Laptop GPU is available.


In [2]:
en_dataset = PropagandaDataset("../data", languages=[Language.EN], tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased'))
pt_dataset = PropagandaDataset("../data", languages=[Language.PT], tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased'))
hi_dataset = PropagandaDataset("../data", languages=[Language.HI], tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased'))
bg_dataset = PropagandaDataset("../data", languages=[Language.BG], tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased'))
all_dataset = PropagandaDataset("../data", languages=[Language.EN, Language.PT, Language.HI, Language.BG], tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased'))

en_train, en_val, en_test = torch.utils.data.random_split(en_dataset, [100, 50, 50])
pt_train, pt_val, pt_test = torch.utils.data.random_split(pt_dataset, [100, 50, 50])
hi_train, hi_val, hi_test = torch.utils.data.random_split(hi_dataset, [60, 25, 30])
bg_train, bg_val, bg_test = torch.utils.data.random_split(bg_dataset, [100, 60, 51])
all_train, all_val, all_test = torch.utils.data.random_split(all_dataset, [400, 163, 163])

datasets = [
    ("EN", "EN", en_train, en_val, en_test),
    ("EN", "ALL", all_train, all_val, en_test),
    ("PT", "PT", pt_train, pt_val, pt_test),
    ("PT", "ALL", all_train, all_val, pt_test),
    ("HI", "HI", hi_train, hi_val, hi_test),
    ("HI", "ALL", all_train, all_val, hi_test),
    ("BG", "BG", bg_train, bg_val, bg_test),
    ("BG", "ALL", all_train, all_val, bg_test),
]


200it [00:00, 9542.59it/s]


Language.EN LabelLevel.NARATIVES (200, 23)
(200, 23)


200it [00:00, 9442.70it/s]


Language.PT LabelLevel.NARATIVES (200, 23)
(200, 23)


115it [00:00, 11751.90it/s]


...

Language.PT LabelLevel.NARATIVES (200, 23)


115it [00:00, 13121.46it/s]


Language.HI LabelLevel.NARATIVES (115, 23)


211it [00:00, 22263.54it/s]


Language.BG LabelLevel.NARATIVES (211, 23)
(726, 23)


In [3]:
from torch import nn
class CustomBertClassifier(torch.nn.Module):
    def __init__(self, hidden_size, out_size):
        super(CustomBertClassifier, self).__init__()
        self.bert = DistilBertModel.from_pretrained(
            "distilbert-base-multilingual-cased", 
        )
        self.dropout1 = nn.Dropout()
        self.linear1 = nn.Linear(in_features=768, out_features=hidden_size, bias=False)
        self.batch_norm1 = nn.BatchNorm1d(num_features=512)
        self.relu1 =  nn.ReLU()
        self.dropout2 = nn.Dropout(p=0.8)
        self.linear2 = nn.Linear(in_features=hidden_size, out_features=out_size, bias=False)
        self.batch_norm2 = nn.BatchNorm1d(num_features=512)
        self.sigmoid = nn.Sigmoid()

    def forward(self, attention_mask, input_ids):
        x = self.bert(attention_mask, input_ids)[0]
        x = self.dropout1(x)
        x = self.linear1(x)
        x = self.batch_norm1(x)
        x = self.relu1(x)
        x = self.dropout2(x)
        x = self.linear2(x)
        x = self.batch_norm2(x)
        x = torch.max(x,1, keepdim=False).values
        return self.sigmoid(x)
    
    def freeze_bert(self):
        """
        Freezes the parameters of BERT so when BertWithCustomNNClassifier is trained
        only the wieghts of the custom classifier are modified.
        """
        for param in self.bert.named_parameters():
            param[1].requires_grad=False
    
    def unfreeze_bert(self):
        """
        Unfreezes the parameters of BERT so when BertWithCustomNNClassifier is trained
        both the wieghts of the custom classifier and of the underlying BERT are modified.
        """
        for param in self.bert.named_parameters():
            param[1].requires_grad=True

    def freeze_embeddings(self):
        """
        Freezes the parameters of BERT so when BertWithCustomNNClassifier is trained
        only the wieghts of the custom classifier are modified.
        """
        for param in self.bert.embeddings.named_parameters():
            param[1].requires_grad=False
    
    def unfreeze_embeddings(self):
        """
        Unfreezes the parameters of BERT so when BertWithCustomNNClassifier is trained
        both the wieghts of the custom classifier and of the underlying BERT are modified.
        """
        for param in self.bert.embeddings.named_parameters():
            param[1].requires_grad=True

In [4]:
from tqdm import tqdm
from torch.optim import Adam
import wandb

def training_step(dataloader, model, optimizer, loss_fn, device, freeze_bert = False, freeze_embeddings = False, runner = None):
    """Method to train the model"""
    model.train()
    model.freeze_bert() if freeze_bert else model.unfreeze_bert()
    model.freeze_embeddings() if not freeze_bert and freeze_embeddings else model.unfreeze_bert()
      
    epoch_loss = 0
    model.to(device)

 
    for x, y in dataloader:
        attention_mask = x["attention_mask"]
        attention_mask = attention_mask.to(device)
        input_ids = x["input_ids"]
        input_ids = input_ids.to(device)
        y = y.to(device)

        output = model(attention_mask, input_ids)
        optimizer.zero_grad()
        loss = loss_fn(output, y.float())
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
        loss_val = loss.item()
        if runner is not None:
            runner.log({"train/loss": loss_val})
        return loss_val

def eval_step(dataloader, model, loss_fn, runner = None):
    """Method to eval the model"""
    model.eval()
    epoch_loss = []
    
    
    with torch.no_grad():
        for x, y in dataloader:
            attention_mask = x["attention_mask"]
            attention_mask = attention_mask.to(device)
            input_ids = x["input_ids"]
            input_ids = input_ids.to(device)        
            output = model(attention_mask, input_ids)
            y = y.to(device)
            loss = loss_fn(output, y.float())
            loss_val = loss.item()
            epoch_loss.append(loss_val)
            if runner is not None:
                runner.log({"eval/loss": loss_val})
    return torch.mean(torch.tensor(epoch_loss)).item()

def eval_model(dataloader, model, scoring_functions):
    outputs = []
    ys = []
    with torch.no_grad():
        for x,y in dataloader:
            attention_mask = x["attention_mask"]
            attention_mask = attention_mask.to(device)
            input_ids = x["input_ids"]
            input_ids = input_ids.to(device)
            output = model(attention_mask, input_ids).cpu()
            outputs.append(output)
            ys.append(y)
    y_pred = torch.concat(outputs).cpu().detach().numpy()
    ys = torch.concat(ys).cpu().float().numpy()
    res = {}
    for sf in scoring_functions:
        name = sf.__name__
        treshold = y_pred > 0.5
        res[name] = sf(treshold, ys, average="macro")

    return res

In [5]:
from sklearn.metrics import f1_score, recall_score, precision_score
scoring_functions = [f1_score, recall_score, precision_score]
lr = 0.0001
batch_size = 32
epochs = 15

all_res = []


for validation_language, train_language, train, val, test in datasets:
    res = {
        "train_language": train_language,
        "validation_language": validation_language
    }
    print(validation_language, train_language)
    torch.cuda.empty_cache()
    torch.cuda.memory_allocated()
    model = CustomBertClassifier(256, 22)
    torch.cuda.empty_cache()
    torch.cuda.memory_allocated()
    optimizer = Adam(model.parameters(), lr=lr)
    loss_fn = loss_fn = torch.nn.BCELoss()
    train_dl = DataLoader(train, batch_size=batch_size)
    eval_dl = DataLoader(val, batch_size=batch_size)
    test_dl = DataLoader(test, batch_size=batch_size)

    best_loss = torch.inf
    for i in range(epochs):
        loss = training_step(train_dl, model, optimizer, loss_fn, device=device, freeze_bert=False)
        eval_loss = eval_step(eval_dl, model, loss_fn)
        print(f"EPOCH[{i}]Train Loss: {loss}, Eval Loss: {eval_loss}")
    res.update(eval_model(test_dl, model, scoring_functions))
    print(res)
    all_res.append(res)
    
    del model
    del train
    del val
    del test


EN EN
EPOCH[0]Train Loss: 2.8571126461029053, Eval Loss: 0.7290617227554321
EPOCH[1]Train Loss: 2.5618772506713867, Eval Loss: 0.7436476945877075
EPOCH[2]Train Loss: 2.376253128051758, Eval Loss: 0.7646613717079163
EPOCH[3]Train Loss: 2.225637197494507, Eval Loss: 0.7813562154769897
EPOCH[4]Train Loss: 2.1178181171417236, Eval Loss: 0.7996973991394043
EPOCH[5]Train Loss: 2.0516104698181152, Eval Loss: 0.8145236968994141
EPOCH[6]Train Loss: 1.9927847385406494, Eval Loss: 0.8250979781150818
EPOCH[7]Train Loss: 1.9506888389587402, Eval Loss: 0.8328666687011719
EPOCH[8]Train Loss: 1.9436171054840088, Eval Loss: 0.8401534557342529
EPOCH[9]Train Loss: 1.8963661193847656, Eval Loss: 0.8456176519393921
EPOCH[10]Train Loss: 1.882435917854309, Eval Loss: 0.8487753868103027
EPOCH[11]Train Loss: 1.8503303527832031, Eval Loss: 0.8518118858337402
EPOCH[12]Train Loss: 1.8476767539978027, Eval Loss: 0.8517365455627441
EPOCH[13]Train Loss: 1.8253920078277588, Eval Loss: 0.8493432998657227
EPOCH[14]Trai

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'EN', 'validation_language': 'EN', 'f1_score': 0.08078416580665543, 'recall_score': 0.05, 'precision_score': 0.4090909090909091}
EN ALL
EPOCH[0]Train Loss: 2.817889928817749, Eval Loss: 0.7406737804412842
EPOCH[1]Train Loss: 2.4836792945861816, Eval Loss: 0.7588606476783752
EPOCH[2]Train Loss: 2.246307134628296, Eval Loss: 0.7811217904090881
EPOCH[3]Train Loss: 2.1307270526885986, Eval Loss: 0.798541784286499
EPOCH[4]Train Loss: 2.035081148147583, Eval Loss: 0.8110811710357666
EPOCH[5]Train Loss: 2.0007641315460205, Eval Loss: 0.8223889470100403
EPOCH[6]Train Loss: 1.9639067649841309, Eval Loss: 0.8335230946540833
EPOCH[7]Train Loss: 1.9284894466400146, Eval Loss: 0.8414144515991211
EPOCH[8]Train Loss: 1.9199912548065186, Eval Loss: 0.8465810418128967
EPOCH[9]Train Loss: 1.8839174509048462, Eval Loss: 0.8502606749534607
EPOCH[10]Train Loss: 1.8630732297897339, Eval Loss: 0.8524623513221741
EPOCH[11]Train Loss: 1.8519270420074463, Eval Loss: 0.8519284725189209
EPOCH[1

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'ALL', 'validation_language': 'EN', 'f1_score': 0.0514944583815429, 'recall_score': 0.028181818181818186, 'precision_score': 0.4090909090909091}
PT PT
EPOCH[0]Train Loss: 2.926715850830078, Eval Loss: 0.7377384305000305
EPOCH[1]Train Loss: 2.5716183185577393, Eval Loss: 0.7512267827987671
EPOCH[2]Train Loss: 2.3172950744628906, Eval Loss: 0.770724892616272
EPOCH[3]Train Loss: 2.2059032917022705, Eval Loss: 0.7885181903839111
EPOCH[4]Train Loss: 2.1067214012145996, Eval Loss: 0.8059723377227783
EPOCH[5]Train Loss: 2.041195869445801, Eval Loss: 0.8222364187240601
EPOCH[6]Train Loss: 2.0055975914001465, Eval Loss: 0.8349993228912354
EPOCH[7]Train Loss: 1.9618964195251465, Eval Loss: 0.84571373462677
EPOCH[8]Train Loss: 1.932041883468628, Eval Loss: 0.8533364534378052
EPOCH[9]Train Loss: 1.8835645914077759, Eval Loss: 0.8590888977050781
EPOCH[10]Train Loss: 1.8852324485778809, Eval Loss: 0.8614515066146851
EPOCH[11]Train Loss: 1.8627675771713257, Eval Loss: 0.86132335662

...

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'ALL', 'validation_language': 'HI', 'f1_score': 0.08335480286489429, 'recall_score': 0.05, 'precision_score': 0.36363636363636365}
BG BG
EPOCH[0]Train Loss: 2.970127582550049, Eval Loss: 0.7420029640197754
EPOCH[1]Train Loss: 2.7165896892547607, Eval Loss: 0.7529512643814087
EPOCH[2]Train Loss: 2.488114595413208, Eval Loss: 0.7663761973381042
EPOCH[3]Train Loss: 2.3250808715820312, Eval Loss: 0.7825716733932495
EPOCH[4]Train Loss: 2.233675956726074, Eval Loss: 0.7995549440383911
EPOCH[5]Train Loss: 2.1504600048065186, Eval Loss: 0.8180508613586426
EPOCH[6]Train Loss: 2.0887632369995117, Eval Loss: 0.83526211977005
EPOCH[7]Train Loss: 2.027958393096924, Eval Loss: 0.849500834941864
EPOCH[8]Train Loss: 2.002443313598633, Eval Loss: 0.8624799251556396
EPOCH[9]Train Loss: 1.9780458211898804, Eval Loss: 0.8699474334716797
EPOCH[10]Train Loss: 1.936424970626831, Eval Loss: 0.8769748210906982
EPOCH[11]Train Loss: 1.9004610776901245, Eval Loss: 0.8811426162719727
EPOCH[12]Tr

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'BG', 'validation_language': 'BG', 'f1_score': 0.09312291505744913, 'recall_score': 0.054367201426024955, 'precision_score': 0.45454545454545453}
BG ALL
EPOCH[0]Train Loss: 2.8997504711151123, Eval Loss: 0.7446050643920898
EPOCH[1]Train Loss: 2.5918376445770264, Eval Loss: 0.7583915591239929
EPOCH[2]Train Loss: 2.3741538524627686, Eval Loss: 0.7793488502502441
EPOCH[3]Train Loss: 2.217302083969116, Eval Loss: 0.8013672828674316
EPOCH[4]Train Loss: 2.1159183979034424, Eval Loss: 0.820798397064209
EPOCH[5]Train Loss: 2.0295653343200684, Eval Loss: 0.8398236632347107
EPOCH[6]Train Loss: 1.9842883348464966, Eval Loss: 0.8549456000328064
EPOCH[7]Train Loss: 1.9517439603805542, Eval Loss: 0.8664462566375732
EPOCH[8]Train Loss: 1.9291025400161743, Eval Loss: 0.8762837052345276
EPOCH[9]Train Loss: 1.9055821895599365, Eval Loss: 0.88286954164505
EPOCH[10]Train Loss: 1.854971170425415, Eval Loss: 0.8860445022583008
EPOCH[11]Train Loss: 1.8590831756591797, Eval Loss: 0.88832896

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

In [6]:
import pandas as pd
pd.DataFrame(all_res).to_csv("raw_epochs_15.csv", index=False)

In [7]:

from sklearn.metrics import f1_score, recall_score, precision_score
scoring_functions = [f1_score, recall_score, precision_score]
lr = 0.0001
batch_size = 32
epochs = 3

all_res = []


for validation_language, train_language, train, val, test in datasets:
    res = {
        "train_language": train_language,
        "validation_language": validation_language
    }
    print(validation_language, train_language)
    torch.cuda.empty_cache()
    torch.cuda.memory_allocated()
    model = CustomBertClassifier(256, 22)
    torch.cuda.empty_cache()
    torch.cuda.memory_allocated()
    optimizer = Adam(model.parameters(), lr=lr)
    loss_fn = loss_fn = torch.nn.BCELoss()
    train_dl = DataLoader(train, batch_size=batch_size)
    eval_dl = DataLoader(val, batch_size=batch_size)
    test_dl = DataLoader(test, batch_size=batch_size)

    best_loss = torch.inf
    for i in range(epochs):
        loss = training_step(train_dl, model, optimizer, loss_fn, device=device, freeze_bert=False)
        eval_loss = eval_step(eval_dl, model, loss_fn)
        print(f"EPOCH[{i}]Train Loss: {loss}, Eval Loss: {eval_loss}")
    res.update(eval_model(test_dl, model, scoring_functions))
    print(res)
    all_res.append(res)
    
    del model
    del train
    del val
    del test


EN EN
EPOCH[0]Train Loss: 2.872985363006592, Eval Loss: 0.7406224012374878
EPOCH[1]Train Loss: 2.54636549949646, Eval Loss: 0.752669095993042
EPOCH[2]Train Loss: 2.316941738128662, Eval Loss: 0.7644317746162415


/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'EN', 'validation_language': 'EN', 'f1_score': 0.060300954030143815, 'recall_score': 0.03272727272727272, 'precision_score': 0.5}
EN ALL
EPOCH[0]Train Loss: 2.949291229248047, Eval Loss: 0.7468878626823425
EPOCH[1]Train Loss: 2.618835687637329, Eval Loss: 0.7538485527038574
EPOCH[2]Train Loss: 2.3437466621398926, Eval Loss: 0.7645307183265686


/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'ALL', 'validation_language': 'EN', 'f1_score': 0.09305620749046183, 'recall_score': 0.05636363636363637, 'precision_score': 0.5909090909090909}
PT PT
EPOCH[0]Train Loss: 2.841423511505127, Eval Loss: 0.7418946027755737
EPOCH[1]Train Loss: 2.4976775646209717, Eval Loss: 0.7562615871429443
EPOCH[2]Train Loss: 2.310530662536621, Eval Loss: 0.7713890075683594


...

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'ALL', 'validation_language': 'HI', 'f1_score': 0.0866372608760913, 'recall_score': 0.051515151515151514, 'precision_score': 0.45454545454545453}
BG BG
EPOCH[0]Train Loss: 2.882192611694336, Eval Loss: 0.7203267216682434
EPOCH[1]Train Loss: 2.6298887729644775, Eval Loss: 0.736505389213562
EPOCH[2]Train Loss: 2.391648054122925, Eval Loss: 0.7567790746688843


/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'BG', 'validation_language': 'BG', 'f1_score': 0.09896880692673612, 'recall_score': 0.057932263814616754, 'precision_score': 0.5909090909090909}
BG ALL
EPOCH[0]Train Loss: 2.791367292404175, Eval Loss: 0.7400941252708435
EPOCH[1]Train Loss: 2.4593043327331543, Eval Loss: 0.755237340927124
EPOCH[2]Train Loss: 2.2524526119232178, Eval Loss: 0.7676727771759033
{'train_language': 'ALL', 'validation_language': 'BG', 'f1_score': 0.10361272972005221, 'recall_score': 0.060606060606060615, 'precision_score': 0.5909090909090909}


/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

In [8]:
pd.DataFrame(all_res).to_csv("raw_epochs_3.csv", index=False)

In [5]:

from sklearn.metrics import f1_score, recall_score, precision_score
scoring_functions = [f1_score, recall_score, precision_score]
lr = 0.0001
batch_size = 32
epochs = 25

all_res = []


for validation_language, train_language, train, val, test in datasets:
    res = {
        "train_language": train_language,
        "validation_language": validation_language
    }
    print(validation_language, train_language)
    torch.cuda.empty_cache()
    torch.cuda.memory_allocated()
    model = CustomBertClassifier(256, 22)
    torch.cuda.empty_cache()
    torch.cuda.memory_allocated()
    optimizer = Adam(model.parameters(), lr=lr)
    loss_fn = loss_fn = torch.nn.BCELoss()
    train_dl = DataLoader(train, batch_size=batch_size)
    eval_dl = DataLoader(val, batch_size=batch_size)
    test_dl = DataLoader(test, batch_size=batch_size)

    best_loss = torch.inf
    for i in range(epochs):
        loss = training_step(train_dl, model, optimizer, loss_fn, device=device, freeze_bert=False)
        eval_loss = eval_step(eval_dl, model, loss_fn)
        print(f"EPOCH[{i}]Train Loss: {loss}, Eval Loss: {eval_loss}")
    res.update(eval_model(test_dl, model, scoring_functions))
    print(res)
    all_res.append(res)
    
    del model
    del train
    del val
    del test


EN EN
EPOCH[0]Train Loss: 2.802311897277832, Eval Loss: 0.7234396934509277
EPOCH[1]Train Loss: 2.4570364952087402, Eval Loss: 0.7374353408813477
EPOCH[2]Train Loss: 2.2248499393463135, Eval Loss: 0.7555204629898071
EPOCH[3]Train Loss: 2.08656644821167, Eval Loss: 0.7727371454238892
EPOCH[4]Train Loss: 2.0150890350341797, Eval Loss: 0.7889270782470703
EPOCH[5]Train Loss: 1.9565988779067993, Eval Loss: 0.8021253347396851
EPOCH[6]Train Loss: 1.918536901473999, Eval Loss: 0.8125678300857544
EPOCH[7]Train Loss: 1.8820964097976685, Eval Loss: 0.8185646533966064
EPOCH[8]Train Loss: 1.8742117881774902, Eval Loss: 0.8228611946105957
EPOCH[9]Train Loss: 1.8434897661209106, Eval Loss: 0.8269164562225342
EPOCH[10]Train Loss: 1.8344078063964844, Eval Loss: 0.8295308351516724
EPOCH[11]Train Loss: 1.8164072036743164, Eval Loss: 0.8306201696395874
EPOCH[12]Train Loss: 1.8098500967025757, Eval Loss: 0.8290721774101257
EPOCH[13]Train Loss: 1.7910906076431274, Eval Loss: 0.8249071836471558
EPOCH[14]Train

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'EN', 'validation_language': 'EN', 'f1_score': 0.08409700705655249, 'recall_score': 0.05090909090909091, 'precision_score': 0.5}
EN ALL
EPOCH[0]Train Loss: 2.9020252227783203, Eval Loss: 0.732322633266449
EPOCH[1]Train Loss: 2.427884101867676, Eval Loss: 0.7548546195030212
EPOCH[2]Train Loss: 2.20741605758667, Eval Loss: 0.7815110087394714
EPOCH[3]Train Loss: 2.058140993118286, Eval Loss: 0.8074036240577698
EPOCH[4]Train Loss: 1.9775540828704834, Eval Loss: 0.8270359635353088
EPOCH[5]Train Loss: 1.9390735626220703, Eval Loss: 0.8413333296775818
EPOCH[6]Train Loss: 1.9110163450241089, Eval Loss: 0.8530659675598145
EPOCH[7]Train Loss: 1.8667831420898438, Eval Loss: 0.8625457882881165
EPOCH[8]Train Loss: 1.8263540267944336, Eval Loss: 0.8690605163574219
EPOCH[9]Train Loss: 1.8286417722702026, Eval Loss: 0.8757278323173523
EPOCH[10]Train Loss: 1.8074218034744263, Eval Loss: 0.8772295117378235
EPOCH[11]Train Loss: 1.7847665548324585, Eval Loss: 0.8788194060325623
EPOCH[12

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarn

{'train_language': 'ALL', 'validation_language': 'EN', 'f1_score': 0.08614668152002006, 'recall_score': 0.052727272727272734, 'precision_score': 0.4090909090909091}
PT PT
EPOCH[0]Train Loss: 2.927497148513794, Eval Loss: 0.7411784529685974
EPOCH[1]Train Loss: 2.5808982849121094, Eval Loss: 0.7430145740509033
EPOCH[2]Train Loss: 2.334838390350342, Eval Loss: 0.750051736831665
EPOCH[3]Train Loss: 2.177323579788208, Eval Loss: 0.7592847347259521
EPOCH[4]Train Loss: 2.0916786193847656, Eval Loss: 0.7709116339683533
EPOCH[5]Train Loss: 2.025998115539551, Eval Loss: 0.7814584970474243
EPOCH[6]Train Loss: 1.973921537399292, Eval Loss: 0.7902335524559021
EPOCH[7]Train Loss: 1.9329774379730225, Eval Loss: 0.7988959550857544
EPOCH[8]Train Loss: 1.8995282649993896, Eval Loss: 0.8065656423568726
EPOCH[9]Train Loss: 1.8737157583236694, Eval Loss: 0.8119115829467773
EPOCH[10]Train Loss: 1.8502901792526245, Eval Loss: 0.8133693933486938
EPOCH[11]Train Loss: 1.8258684873580933, Eval Loss: 0.8136513233

...

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'train_language': 'BG', 'validation_language': 'BG', 'f1_score': 0.08657723907035508, 'recall_score': 0.051693404634581115, 'precision_score': 0.5}
BG ALL
EPOCH[0]Train Loss: 2.9255683422088623, Eval Loss: 0.7431294322013855
EPOCH[1]Train Loss: 2.5846827030181885, Eval Loss: 0.7490301728248596
EPOCH[2]Train Loss: 2.380258798599243, Eval Loss: 0.7613243460655212
EPOCH[3]Train Loss: 2.246845006942749, Eval Loss: 0.776584804058075
EPOCH[4]Train Loss: 2.1525394916534424, Eval Loss: 0.7883837819099426
EPOCH[5]Train Loss: 2.0833404064178467, Eval Loss: 0.7993783950805664
EPOCH[6]Train Loss: 2.034374952316284, Eval Loss: 0.8098183274269104
EPOCH[7]Train Loss: 1.9811780452728271, Eval Loss: 0.8213204741477966
EPOCH[8]Train Loss: 1.9541876316070557, Eval Loss: 0.8311701416969299
EPOCH[9]Train Loss: 1.929803729057312, Eval Loss: 0.8387600779533386
EPOCH[10]Train Loss: 1.8892097473144531, Eval Loss: 0.8435528874397278
EPOCH[11]Train Loss: 1.873854398727417, Eval Loss: 0.8485427498817444
EPOCH[12

/home/itsv.org.sv-services.at/tibor.cus@itsv.at/.cache/pypoetry/virtualenvs/toxicml-_rQSQEXi-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'train_language': 'ALL', 'validation_language': 'BG', 'f1_score': 0.09638973195013065, 'recall_score': 0.0552584670231729, 'precision_score': 0.5909090909090909}


NameError: name 'pd' is not defined

In [6]:
import pandas as pd
pd.DataFrame(all_res).to_csv("raw_epochs_25.csv", index=False)
